In [1]:
import sys

sys.path.append("../src")

### Load Q&A test data

In [3]:
!mc cp s3/projet-llm-insee-open-data/data/eval_data/eval_retrieval/q_and_a_scored_filtered_Phi-3-mini-128k-instruct.csv ../data/q_and_a_ref_retrieval_evaluation_Phi-3-mini-128k-instruct.csv

...struct.csv: 281.66 KiB / 281.66 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 555.67 KiB/s 0s

### Load Knowledge Data

In [9]:
!mc cp s3/projet-llm-insee-open-data/data/eval_data/eval_retrieval/insee_documents_sample_ref_retrieval_evaluation.csv ../data/insee_documents_sample_ref_retrieval_evaluation.csv

...uation.csv: 4.69 MiB / 4.69 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 8.54 MiB/s 0s

In [2]:
import pandas as pd 

#question and answer
path_qa = "../data/q_and_a_ref_retrieval_evaluation_Phi-3-mini-128k-instruct.csv"
test = pd.read_csv(path_qa)

df_dict = {}
df_dict["the_df_dataset"] = test

#knowledge data 
path_knowledge = "../data/insee_documents_sample_ref_retrieval_evaluation.csv"
data = pd.read_csv(path_knowledge)

In [3]:
display(test.head())
display(data.head())

,context,question,answer,source_doc,groundedness_score,groundedness_eval,relevance_score,relevance_eval,standalone_score,standalone_eval
0,. L’approche coût-efficacité apparaît plus con...,Quelle est l'approche qui s'aligne sur l'objec...,L'approche coût-efficacité est celle qui s'ali...,https://www.insee.fr/fr/information/4796342,5.0,Le contexte fourni donne des informations sur ...,4.0,Cette question est très pertinente pour les ag...,5.0,La question mentionne explicitement l'accord d...
1,". A contrario, pour les femmes de 30à 34ans, i...",Quelle est l'évolution de l'âge moyen à la mat...,L'âge moyen à la maternité pour les femmes de ...,https://www.insee.fr/fr/statistiques/4238437,5.0,La question demande une évolution spécifique c...,5.0,Cette question est très pertinente pour les ag...,5.0,Cette question est indépendante du contexte ca...
2,. Points d’entrées pour la France dans les coo...,Quelle est la part des mineurs dans les cas de...,La part des mineurs a diminué de 7 points pour...,https://www.insee.fr/fr/statistiques/5763625,4.0,Le contexte fournit des informations détaillée...,4.0,Cette question est très pertinente pour les ag...,4.0,Cette question est assez indépendante du conte...
3,. L’empreinte énergétique directe du numérique...,Quelle est la part du numérique dans les émiss...,"En 2017, le numérique représentait plus de 3,7...",https://www.insee.fr/fr/statistiques/4126596,5.0,Le contexte fourni donne des informations spéc...,5.0,Cette question est très pertinente pour les ag...,4.0,Cette question demande des informations spécif...
4,. La collecte d’Acemo Covid s’étend sur deux ...,Quels sont les codes APE utilisés pour représe...,Les codes APE utilisés pour représenter la res...,https://www.insee.fr/fr/statistiques/5392036,5.0,La question demande de nommer les codes APE co...,4.0,Cette question est directement liée aux activi...,5.0,La question demande une information spécifique...


,content,source,title,insee_id,categories,date_diffusion,themes,collections,libelleAffichageGeo,intertitres,authors,subtitle
0,"La Base tous salariés (BTS), extraite de la Dé...",https://www.insee.fr/fr/information/2407785,Base Tous Salariés,2407785,Services,2023-10-03T12:04:14.154Z,NaN,NaN,NaN,Description des variables des différents fichi...,[],BTS
1,"En France, en mars-avril 2020, un surcroît de ...",https://www.insee.fr/fr/statistiques/4797670,Les inégalités sociales à l’épreuve de la cris...,4797670,Publications grand public,2020-12-03T17:00:25.538Z,"[""Revenus – Niveaux de vie – Pouvoir d'achat""]","France, portrait social",France,Décrire au mieux l’impact de la crise sanitair...,"['Meriam Barhoumi (Depp)', 'Anne Jonchery', 'P...",NaN
2,L'Insee a conçu une base historique des Recens...,https://www.insee.fr/fr/information/1303688,RP Historiques et Données harmonisées des rece...,1303688,Services,2023-04-25T09:00:14.283Z,NaN,NaN,NaN,Description des variables des différents\n ...,[],NaN
3,"Il ne faut pas confondre l'arrondissement, sub...",https://www.insee.fr/fr/information/1303686,Recensement de la population de 2006 à 2020,1303686,Services,2023-09-06T06:37:14.496Z,NaN,NaN,NaN,Description des variables des différents\n ...,[],NaN
4,"En 2018, l’économie française ralentit: le pro...",https://www.insee.fr/fr/statistiques/4174895,"En 2018, l’économie française ralentit, après ...",4174895,Publications grand public,2019-06-28T16:00:00Z,['Évolution et structure de la population'],L'économie française - Comptes et dossiers,France,Figure 1 - Contribution des principaux agrégat...,"['Victor\xa0Barry', 'Thibault Cruzet', 'Domini...",NaN


In [4]:
from evaluation import RetrievalConfiguration, RetrievalEvaluator

conf = RetrievalConfiguration(
        name='test2',
        database="chromadb",
        database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
        collection="test_data",
        embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
        reranker_type=None,
        reranker_name=None,
        rerank_k=5, 
        k_values=[5, 10],

    )

2024-07-01 15:27:13,522 - INFO - Loading faiss with AVX512 support.
2024-07-01 15:27:13,883 - INFO - Successfully loaded faiss with AVX512 support.


In [ ]:
from evaluation import RetrievalConfiguration, RetrievalEvaluator

# Define retrieval configurations
eval_configs = [
    RetrievalConfiguration(
        name='test1',
        database="chromadb",
        database_path="../data/insee_documents_sample_ref_retrieval_evaluation.csv",
        collection="test_data",
        embedding_model_name="OrdalieTech/Solon-embeddings-large-0.1",
        reranker_type='BM25',
        reranker_name=None,
        rerank_k=5, 
        k_values=[5, 10],
    )
]

# Run the evaluator
results = RetrievalEvaluator.run(
    eval_configurations=eval_configs,
    eval_dict=df_dict,  # Ensure 'df_dict' is a dictionary containing pandas DataFrames with the required structure
)

# Print or process results
print(results)


## Test 28/06/2024 - Update Retrieval Evaluation process